### Import library 

In [13]:
import torch
import pandas as pd
import PIL
import matplotlib.pylab as plt

### Visdom

In [2]:
import visdom
import numpy as np
vis = visdom.Visdom(env='ch2')


### Pytorch

In [3]:
import torchvision.transforms as transforms
from torchvision.utils import save_image
from torch.utils.data import DataLoader
from torchvision import datasets,models

import torch.nn as nn
import torch.nn.functional as F
import torch

### Torch Vision

In [4]:
feature_extract = models.densenet121(pretrained=True)

/home/jimmyyoung/.local/lib/python3.5/site-packages/torchvision/models/densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(m.weight.data)


In [5]:
root_list = ['HMB_1/','HMB_2/','HMB_4/','HMB_5/','HMB_6/']

hbm_1_data = pd.read_csv('./HMB_1/interpolated.csv')
hbm_2_data = pd.read_csv('./HMB_2/interpolated.csv')
hbm_4_data = pd.read_csv('./HMB_4/interpolated.csv')
hbm_5_data = pd.read_csv('./HMB_5/interpolated.csv')

hbm_6_data = pd.read_csv('./HMB_6/interpolated.csv')
df_list = [hbm_1_data,hbm_2_data,hbm_4_data,hbm_5_data,hbm_6_data]
df_center_list = []
for i,df in enumerate(df_list):
    a = df.groupby('frame_id').get_group('center_camera')
    a.sort_values('timestamp')
    a.filename = root_list[i]+a.filename
    df_center_list.append(a)
    

/usr/local/lib/python3.5/dist-packages/pandas/core/generic.py:4401: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [6]:
video = []
for i in range(100):
    video.append(plt.imread(df_center_list[1].iloc[i].filename))
video = np.array(video)

In [7]:
vis.video(video)


'window_36c7946cf95d56'

In [8]:
class PARSER():
    def __init__(self):
        self.n_epochs = 300
        self.batch_size = 64
        self.lr = 0.0001
        self.b1 = 0.5
        self.b2 = 0.999
        self.n_cpu = 8
        self.latent_dim = 62
        self.img_size = 64
        self.channels = 1
        self.sample_interval = 1000
        self.clip_value = 0.01
opt = PARSER()




### Cuda

In [9]:
cuda = True if torch.cuda.is_available() else False
device = 'cuda' if cuda else 'cpu'


### Weight Norm

In [10]:
def weights_init_normal(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        torch.nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm2d') != -1:
        torch.nn.init.normal_(m.weight.data, 1.0, 0.02)
        torch.nn.init.constant_(m.bias.data, 0.0)

In [11]:
# Policy Network
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.conv1_0 = nn.Sequential(
            nn.Conv3d(3, 64, (3, 3, 3), stride=(1, 2, 2), padding=(0, 1, 1)),
            nn.ReLU(),
        )
        self.maxpool1_0 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
        
        self.conv2 = nn.Sequential(
            feature_extract.features.denseblock1,
            feature_extract.features.transition1,
            feature_extract.features.denseblock2,
            feature_extract.features.transition2,
            feature_extract.features.denseblock3,
            feature_extract.features.transition3,
            
        )
        self.maxpool2_0 = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Linear(in_features=512,out_features=3)
        
#         self.conv4 = nn.Conv2d(32,1,3,1,1)
            
    def forward(self, x):
        
        x = F.relu(self.conv1_0(x)) 
        x = x.squeeze(2)
        x = self.maxpool1_0(x)
        x = self.conv2(x)
        x = self.maxpool2_0(x)
        x = F.tanh(self.fc(x.view(-1,512)))
#         x = F.sigmoid(self.conv4(x))
        return x
# Reward function   
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        
        self.conv = nn.Sequential(
            nn.Conv2d(4,32,3,1,1),
            nn.BatchNorm2d(32),
            nn.LeakyReLU(),
            
            nn.Conv2d(32,32,3,1,1),
            nn.BatchNorm2d(32),
            nn.LeakyReLU(),
            
            nn.Conv2d(32,32,3,2,1),
            nn.BatchNorm2d(32),
            nn.LeakyReLU(),
            
            nn.Conv2d(32,32,3,2,1),
            nn.BatchNorm2d(32),
            nn.LeakyReLU(),
            
            nn.Conv2d(32,32,3,2,1),
            nn.BatchNorm2d(32),
            nn.LeakyReLU(),
            
        )
        self.fc = nn.Linear(8*8*32,1)
        self.Htanh = torch.nn.Hardtanh()
        

    def forward(self, img):
        out = self.conv(img)
        
        out = self.fc(out.view(-1,8*8*32))
         
        return self.Htanh(out)

    
Policy_net = Generator()
Reward_net = Discriminator()
weights_init_normal(Policy_net)
weights_init_normal(Reward_net)
Policy_net,Reward_net = Policy_net.to(device),Reward_net.to(device)

### Transform

In [14]:
transform = transforms.Compose([
    transforms.RandomAffine(5),
    transforms.ColorJitter(0.1,0.1,0.1,0.1),
    transforms.ToTensor(),
    
])

### Sample image randomly for five image

In [34]:
img_list = []
for i in range(3):
    a = PIL.Image.open(df_center_list[0].iloc[i].filename)
    img_list.append(transform(a))
img_list = torch.stack(img_list).unsqueeze(0)

In [37]:
Policy_net(img_list.to(device))

torch.Size([1, 512, 1, 1])


/home/jimmyyoung/.local/lib/python3.5/site-packages/torch/nn/functional.py:995: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


tensor([[ 0.0286, -0.0245,  0.0019]], device='cuda:0', grad_fn=<TanhBackward>)

### Sample

In [21]:
vis.image(transform(PIL.Image.open(df_center_list[0].iloc[0].filename)))

'window_36c79c67e9f818'

In [12]:
Policy_net(torch.ones((1,3,3,480,640)).to(device))

torch.Size([1, 512, 1, 1])


/home/jimmyyoung/.local/lib/python3.5/site-packages/torch/nn/functional.py:995: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


tensor([[ 0.0355, -0.0231, -0.0114]], device='cuda:0', grad_fn=<TanhBackward>)

### Training Progress
random sample continuous 3 images and predict the next frame